In [9]:
## ### Preprocess of 'RED VALENCIANA DE BIOBANCOS' records (RVB samples) ###############
# Import libraries and read clinical file
import pandas as pd
import numpy as np
rvb_path='/home/vant/TFM/clinical data/RedValencianaBiobancos_RVB_series_Lynch/2020910_ Base Datos Clínicos solicitados RVB codif_ID CNIO.xls'
rvb=pd.read_excel(rvb_path,engine='xlrd')
# Show first rows and print the length
print(rvb.head(9))
print("RVB cohort file has a length of:", len(rvb))

  ID CNIO_MJG Código de envío  NAME (INITIALS)  ID (Clinical History Number)  \
0         NaN             NaN              NaN                           NaN   
1        RVB1           28401              NaN                           NaN   
2        RVB2           28402              NaN                           NaN   
3        RVB3           28403              NaN                           NaN   
4        RVB4           28404              NaN                           NaN   
5        RVB5           28405              NaN                           NaN   
6        RVB6           28406              NaN                           NaN   
7        RVB7           28407              NaN                           NaN   
8        RVB8           28408              NaN                           NaN   

                 DATE OF BIRTH                             DATE OF DIAGNOSIS  \
0  ( DD-MM-YYYY; NA=DON´T KNOW)                                   DD-MM-YYYY   
1           1968-03-14 00:00:00        

In [10]:
## Print information about the dataframe created
print(rvb.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 83 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   ID CNIO_MJG                                                                    9 non-null      object 
 1   Código de envío                                                                10 non-null     object 
 2   NAME (INITIALS)                                                                0 non-null      float64
 3   ID (Clinical History Number)                                                   0 non-null      float64
 4   DATE OF BIRTH                                                                  10 non-null     object 
 5   DATE OF DIAGNOSIS                                                              10 non-null     object 
 6   DATE OF DIAGNOSIS_OVARIAN CA

In [11]:
# Clean column names
print('Colums of RVB before eliminating spaces are:', rvb.columns)
rvb.columns = rvb.columns.str.strip()
print('Columns of RVB after eliminating spaces on the right and left:',rvb.columns)

Colums of RVB before eliminating spaces are: Index(['ID CNIO_MJG', 'Código de envío', 'NAME (INITIALS)',
       'ID (Clinical History Number)', 'DATE OF BIRTH ', 'DATE OF DIAGNOSIS',
       'DATE OF DIAGNOSIS_OVARIAN CANCER', 'AGE AT DIAGNOSIS', 'SERIES TYPE',
       'Tissue ID (Pathology Report Number)', 'HISTOLOGY',
       'HISTOLOGY_coded MJG', 'BEHAVIOUR', 'PATHOLOGIC REVIEW',
       'NORMAL TISSUE', 'NORMAL TISSUE source', 'FIGO STAGE', 'FIGO STAGE_MJG',
       'TUMOR GRADE ( two tier system, Malpica et al.  ONLY SEROUS SUBTYPE)',
       'TUMOR GRADE', 'PRESENCE OF TUMOUR INFILTRATING LYMPHOCYTES',
       'PRESENCE OF TUMOUR INFILTRATING LYMPHOCYTES AT INVASIVE TUMOR MARGIN',
       'EVALUATION OF TUMOUR INFILTRATING LUMPHOCYTES',
       'CD8 IHC STAINING intraepithelial', 'CD8 IHC STAINING intrastromal',
       'CD8 IHC STAINING invasive margin', 'CD3 IHC STAINING intraepithelial',
       'CD3 IHC STAINING intrastromal', 'CD3 IHC STAINING invasive margin',
       'PD-L1 IHC STAIN

In [12]:
#### Keep all rows until the first empty one  ############
# Find index of the first empty row
first_empty_row = rvb[rvb.isnull().all(axis=1)].index[0] # index[0] refers to the first index for null data
print('The first empty row in the file has index:',first_empty_row)
# Keep row data until the first empty row
rvb = rvb.iloc[:first_empty_row] ## Include the row with index 0 
print("The length/number of rows of the RVB dataframe once eliminated the legends in the end is:", len(rvb)) 
print(rvb)

The first empty row in the file has index: 10
The length/number of rows of the RVB dataframe once eliminated the legends in the end is: 10
  ID CNIO_MJG Código de envío  NAME (INITIALS)  ID (Clinical History Number)  \
0         NaN             NaN              NaN                           NaN   
1        RVB1           28401              NaN                           NaN   
2        RVB2           28402              NaN                           NaN   
3        RVB3           28403              NaN                           NaN   
4        RVB4           28404              NaN                           NaN   
5        RVB5           28405              NaN                           NaN   
6        RVB6           28406              NaN                           NaN   
7        RVB7           28407              NaN                           NaN   
8        RVB8           28408              NaN                           NaN   
9        RVB9           28409              NaN               

In [13]:
# In 'TUMOR GRADE', 'MMR GERMLINE STATUS' and 'VITAL STATUS' columns I have to keep only the first element before =.
print(rvb[['TUMOR GRADE','MMR GERMLINE STATUS','VITAL STATUS']])
# Define the function to work
def extract_before_equals(cell):
    """Extract text before ('=') in each part of the cell"""
    if isinstance(cell, str):
        parts = cell.split(';') # There are some rows with 2 data separated by ; at first
        processed_parts = [part.split('=')[0] for part in parts]
        return ';'.join(processed_parts)
    return cell

# Apply the function starting in the row with index 1 (the first with samples) 
rvb.loc[1:,'TUMOR GRADE'] = rvb.loc[1:,'TUMOR GRADE'].apply(extract_before_equals)
rvb.loc[1:,'MMR GERMLINE STATUS'] = rvb.loc[1:,'MMR GERMLINE STATUS'].apply(extract_before_equals)
rvb.loc[1:,'VITAL STATUS'] = rvb.loc[1:,'VITAL STATUS'].apply(extract_before_equals)
print(rvb[['TUMOR GRADE','MMR GERMLINE STATUS','VITAL STATUS']])

                                         TUMOR GRADE  \
0  1=well differentiated, 2=moderately differenti...   
1                                                NaN   
2                              1=well differentiated   
3                                                NaN   
4  ovario:3=poorly differentiated; ENDOMETRIO: 1=...   
5                              1=well differentiated   
6                                      NA=don´t know   
7                                      NA=don´t know   
8                            3=poorly differentiated   
9                                      NA=don´t know   

                   MMR GERMLINE STATUS         VITAL STATUS  
0  0=UNKNOW, 1=MUTATED, 2= UNMUTATED,   0 = alive, 1 = dead  
1                            1=MUTATED             1 = dead  
2                            1=MUTATED             1 = dead  
3                            1=MUTATED            0 = alive  
4                            1=MUTATED             1 = dead  
5          

In [14]:
#########    Prepare columns for calculating Overall survival (OS) ############################
## Print necessary columns. OS  is calculated as 'DATE OF DEATH OR LAST VISIT' MINUS 'DATE OF DIAGNOSIS_OVARIAN_CANCER' and
## AGE can be calculated using the 'DATE OF BIRTH' column
print(rvb[['ID CNIO_MJG','DATE OF DIAGNOSIS_OVARIAN CANCER','DATE OF DEATH OR LAST VISIT']])
## Issues: Partial dates represented as XX in one of the terms in the date: day/month/year
# I create functions to replace partial dates to allow Overall survival calculation: one for detect partial dates and the other one
# to replace partial dates by complete dates. We decide to replace XX/XX/year dates by 15/06/year and XX/month/year by 15/month/year
# to allow the calculate of an estimated OS in those cases.

def detect_partial(date_str):
    """Detect partial dates"""
    if isinstance(date_str, str) and 'XX' in date_str:
        return 'YES'
    return 'NO'
def patch_date(date_str):
    """Convert to datetime format, considering partial dates"""
    if isinstance(date_str, str) and 'XX' in date_str:
        parts = date_str.split('/')
        if len(parts) == 3:
            # Replace XX/XX/year by 15/06/year
            if parts[0] == 'XX' and parts[1] == 'XX':
                return pd.to_datetime(f'15/06/{parts[2]}', format='%d/%m/%Y')
            # Replace XX/month/year by 01/month/year
            elif parts[0] == 'XX':
                return pd.to_datetime(f'15/{parts[1]}/{parts[2]}', format='%d/%m/%Y')
    # For the rest values
    return pd.to_datetime(date_str, format='%d/%m/%Y', errors='coerce')

# Apply first function to the relevant columns to obtain partial dates information in new tags columns that we add to the dataframe
rvb['PARTIAL DATE DEATH_LASTv'] = rvb['DATE OF DEATH OR LAST VISIT'].apply(detect_partial)
rvb['PARTIAL DATE DIAGNOSIS'] = rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'].apply(detect_partial)

# Correct partial dates
rvb['DATE OF DEATH OR LAST VISIT'] = rvb['DATE OF DEATH OR LAST VISIT'].apply(patch_date)
rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'] = rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'].apply(patch_date)
#rvb['DATE OF BIRTH'] = rvb['DATE OF BIRTH'].apply(patch_date)
print("After being modified:\n",rvb[['ID CNIO_MJG','DATE OF DIAGNOSIS_OVARIAN CANCER','DATE OF DEATH OR LAST VISIT']])

  ID CNIO_MJG DATE OF DIAGNOSIS_OVARIAN CANCER DATE OF DEATH OR LAST VISIT
0         NaN                       DD-MM-YYYY                  DD-MM-YYYY
1        RVB1              2014-04-18 00:00:00         2016-02-08 00:00:00
2        RVB2                       XX/XX/2006         2010-09-20 00:00:00
3        RVB3              2018-03-08 00:00:00         2018-09-25 00:00:00
4        RVB4              2017-05-25 00:00:00         2020-03-12 00:00:00
5        RVB5              2008-12-14 00:00:00         2020-08-06 00:00:00
6        RVB6              2019-01-31 00:00:00         2019-04-08 00:00:00
7        RVB7              2014-02-27 00:00:00         2015-11-12 00:00:00
8        RVB8              2004-02-23 00:00:00         2017-07-17 00:00:00
9        RVB9              2004-03-15 00:00:00         2012-12-24 00:00:00
After being modified:
   ID CNIO_MJG DATE OF DIAGNOSIS_OVARIAN CANCER DATE OF DEATH OR LAST VISIT
0         NaN                              NaT                         NaT
1 

In [15]:
# Convert dates to datetime format
rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'] = pd.to_datetime(rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'], format='%d/%m/%Y',errors='coerce')
rvb['DATE OF DEATH OR LAST VISIT'] = pd.to_datetime(rvb['DATE OF DEATH OR LAST VISIT'], format='%d/%m/%Y',errors='coerce')
# Crete the new column 'OS_CNIO'
rvb['OS_CNIO'] = (rvb['DATE OF DEATH OR LAST VISIT'] - rvb['DATE OF DIAGNOSIS_OVARIAN CANCER']).dt.days
print(rvb[['ID CNIO_MJG','DATE OF DIAGNOSIS_OVARIAN CANCER','DATE OF DEATH OR LAST VISIT','OS_CNIO']])
# Add tag column 'DATE USED FOR OS' that indicates if the OS has been calculated from the diagnosis date (D) or not
rvb['DATE USED FOR OS'] = rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'].apply(lambda x: 'D' if pd.notnull(x) else 'NA')
print(rvb[['DATE OF DIAGNOSIS_OVARIAN CANCER','DATE USED FOR OS']])

  ID CNIO_MJG DATE OF DIAGNOSIS_OVARIAN CANCER DATE OF DEATH OR LAST VISIT  \
0         NaN                              NaT                         NaT   
1        RVB1                       2014-04-18                  2016-02-08   
2        RVB2                       2006-06-15                  2010-09-20   
3        RVB3                       2018-03-08                  2018-09-25   
4        RVB4                       2017-05-25                  2020-03-12   
5        RVB5                       2008-12-14                  2020-08-06   
6        RVB6                       2019-01-31                  2019-04-08   
7        RVB7                       2014-02-27                  2015-11-12   
8        RVB8                       2004-02-23                  2017-07-17   
9        RVB9                       2004-03-15                  2012-12-24   

   OS_CNIO  
0      NaN  
1    661.0  
2   1558.0  
3    201.0  
4   1022.0  
5   4253.0  
6     67.0  
7    623.0  
8   4893.0  
9   3206.0 

In [16]:
# Calculate age at diagnosis and compare with the one provided by the hospital/centre
# Firstly I use the function previously created to patch partial dates
rvb['DATE OF BIRTH'] = rvb['DATE OF BIRTH'].apply(patch_date)
# Convert birth dates to datetime format
rvb['DATE OF BIRTH'] = pd.to_datetime(rvb['DATE OF BIRTH'], format='%d/%m/%Y', errors='coerce')
# Calculate Age at diagnosis using
rvb['Age at diagnosis'] = (rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'] - rvb['DATE OF BIRTH']).dt.total_seconds() / (365.25 * 24 * 60 * 60)
# Round
rvb['Age at diagnosis'] = rvb['Age at diagnosis'].round()
print(rvb[['ID CNIO_MJG','DATE OF DIAGNOSIS_OVARIAN CANCER','DATE OF BIRTH','Age at diagnosis','AGE AT DIAGNOSIS']])
# I decide to keep our calculation 

  ID CNIO_MJG DATE OF DIAGNOSIS_OVARIAN CANCER DATE OF BIRTH  \
0         NaN                              NaT           NaT   
1        RVB1                       2014-04-18    1968-03-14   
2        RVB2                       2006-06-15    1959-09-19   
3        RVB3                       2018-03-08    1982-09-07   
4        RVB4                       2017-05-25    1934-02-05   
5        RVB5                       2008-12-14    1969-08-13   
6        RVB6                       2019-01-31    1965-09-06   
7        RVB7                       2014-02-27    1969-04-17   
8        RVB8                       2004-02-23    1942-03-04   
9        RVB9                       2004-03-15    1960-01-02   

   Age at diagnosis                           AGE AT DIAGNOSIS  
0               NaN  (2 digit age at diagnosis; NA=don´t know)  
1              46.0                        43-colon(46)-ovario  
2              47.0                                   45-colon  
3              35.0                

In [17]:
# Calculate OS_CNIO
rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'] = pd.to_datetime(rvb['DATE OF DIAGNOSIS_OVARIAN CANCER'], format='%d/%m/%Y',errors='coerce')
rvb['DATE OF DEATH OR LAST VISIT'] = pd.to_datetime(rvb['DATE OF DEATH OR LAST VISIT'], format='%d/%m/%Y',errors='coerce')
# Crete the new column OS_CNIO
rvb['OS_CNIO'] = (rvb['DATE OF DEATH OR LAST VISIT'] - rvb['DATE OF DIAGNOSIS_OVARIAN CANCER']).dt.days
print(rvb[['ID CNIO_MJG','DATE OF DIAGNOSIS_OVARIAN CANCER','DATE OF DEATH OR LAST VISIT','OS_CNIO']])

  ID CNIO_MJG DATE OF DIAGNOSIS_OVARIAN CANCER DATE OF DEATH OR LAST VISIT  \
0         NaN                              NaT                         NaT   
1        RVB1                       2014-04-18                  2016-02-08   
2        RVB2                       2006-06-15                  2010-09-20   
3        RVB3                       2018-03-08                  2018-09-25   
4        RVB4                       2017-05-25                  2020-03-12   
5        RVB5                       2008-12-14                  2020-08-06   
6        RVB6                       2019-01-31                  2019-04-08   
7        RVB7                       2014-02-27                  2015-11-12   
8        RVB8                       2004-02-23                  2017-07-17   
9        RVB9                       2004-03-15                  2012-12-24   

   OS_CNIO  
0      NaN  
1    661.0  
2   1558.0  
3    201.0  
4   1022.0  
5   4253.0  
6     67.0  
7    623.0  
8   4893.0  
9   3206.0 

In [18]:
# 'FIGO_STAGE_MJG' contains information about the spread of the tumor (Localized, Advanced) with the coding numbers 1 to 13 that define
# different stages:
#    'IA': 1, 'IB':2, 'IC':3, 'I':4,'IIA':5, 'IIB':6, 
#   'II':8,'IIIA':9, 'IIIB':10, 'IIIC':11, 'III':12, 'IV':13
# I need to create 'FIGOa' and 'FIGOL' columns to obtain columns with summarized information

# FIGOa column:  1= IA, IB, IC, I(NOS) (1 to 4); 2=IIA, IIB, II (NOS) (5 to 8); 3=IIIA, IIIB, IIIC, III(NOS) (9 to 12); 4= IV (13).
# Map FIGO_STAGE_MJG (1-13) to calculate FIGOa. 
figo_stage_map = {
    1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 2, 
    8: 2, 9: 3, 10: 3, 11: 3, 12: 3, 13: 4
}
## Map to obtain the new column values
rvb['FIGOa']=rvb['FIGO STAGE_MJG'].map(figo_stage_map)

# FIGOL binary column: 0='Localized' (includes 1 to 8 FIGO stages),1='Advanced'(includes 9 to 13 FIGO stages))
# Define the function to assign the values to the FIGOL column
def classify_figo_stage(stage):
    try:
        stage = int(stage) # Convert into an integer to avoid errors
        if 1 <= stage <= 8:
            return 0
        elif 9 <= stage <= 13:
            return 1
    except (ValueError, TypeError):
        return 'NA'
    return 'NA'

# Apply the function to create FIGOL
rvb['FIGOL'] = rvb['FIGO STAGE_MJG'].apply(classify_figo_stage)
# Print the columns
print(rvb[['FIGO STAGE_MJG','FIGOa','FIGOL']])

   FIGO STAGE_MJG  FIGOa FIGOL
0             NaN    NaN    NA
1             NaN    NaN    NA
2             1.0    1.0     0
3             NaN    NaN    NA
4             8.0    2.0     0
5             6.0    2.0     0
6             1.0    1.0     0
7             8.0    2.0     0
8             NaN    NaN    NA
9             NaN    NaN    NA


In [19]:
# See columns in the dataframe for next filtering step
print(rvb.columns)

Index(['ID CNIO_MJG', 'Código de envío', 'NAME (INITIALS)',
       'ID (Clinical History Number)', 'DATE OF BIRTH', 'DATE OF DIAGNOSIS',
       'DATE OF DIAGNOSIS_OVARIAN CANCER', 'AGE AT DIAGNOSIS', 'SERIES TYPE',
       'Tissue ID (Pathology Report Number)', 'HISTOLOGY',
       'HISTOLOGY_coded MJG', 'BEHAVIOUR', 'PATHOLOGIC REVIEW',
       'NORMAL TISSUE', 'NORMAL TISSUE source', 'FIGO STAGE', 'FIGO STAGE_MJG',
       'TUMOR GRADE ( two tier system, Malpica et al.  ONLY SEROUS SUBTYPE)',
       'TUMOR GRADE', 'PRESENCE OF TUMOUR INFILTRATING LYMPHOCYTES',
       'PRESENCE OF TUMOUR INFILTRATING LYMPHOCYTES AT INVASIVE TUMOR MARGIN',
       'EVALUATION OF TUMOUR INFILTRATING LUMPHOCYTES',
       'CD8 IHC STAINING intraepithelial', 'CD8 IHC STAINING intrastromal',
       'CD8 IHC STAINING invasive margin', 'CD3 IHC STAINING intraepithelial',
       'CD3 IHC STAINING intrastromal', 'CD3 IHC STAINING invasive margin',
       'PD-L1 IHC STAINING (B7-H1) intraepithelial',
       'PD-L1 IH

In [20]:
# I select the columns I want for the final file (I keep them in a list called header_seleccionados).
headers_seleccionados=['ID CNIO_MJG','HISTOLOGY_coded MJG','Age at diagnosis','TUMOR GRADE','FIGO STAGE_MJG','FIGOa','FIGOL',
                       'FIRST LINE or adjuvant TREATMENT_MJG','First line regimen_MJG','MMR GERMLINE STATUS','VITAL STATUS','OS_CNIO','DATE USED FOR OS',
                       'PARTIAL DATE DEATH_LASTv', 'PARTIAL DATE DIAGNOSIS','FAMILIAL ANTECEDENTS OF  COLON, ENDOMETRIAL AND/OR OVARIAN CANCER']
rvb_final=rvb[headers_seleccionados]
print(rvb_final.head())
# Create a row with the coding for every header
coding_row = pd.DataFrame([{
    'ID CNIO_MJG': 'ID CNIO',
    'HISTOLOGY_coded MJG': '1=serous, 2=mucinous, 3=endometrioid, 4=clear cell, 5=mixed cell, 6=other specified epithelial ovarian cancer (e.g. Brenner), 7=undifferentiated epithelial, NA=don´t know;',
    'Age at diagnosis': 'years',
    'TUMOR GRADE': '1=well differentiated, 2=moderately differentiated, 3=poorly differentiated, NA=unknown',
    'FIGO STAGE_MJG': '1=IA, 2=IB, 3=IC, 4=I(NOS), 5=IIA, 6=IIB, 7=IIC, 8=II(NOS), 9=IIIA, 10=IIIB, 11=IIIC, 12=III(NOS), 13=IV, NA=don´t know',
    'FIGOa':'1,2,3,4 or NA=unknown',
    'FIGOL':'0=Localized,1=Avdvanced',
    'FIRST LINE or adjuvant TREATMENT_MJG':'CHEMOTHERAPY NO=0, YES=1,NA=UNKNOWN',
    'First line regimen_MJG':'0=Carbo-paclitaxel;1=Cis-paclitaxel iv;2=Cis-pacli IP;3=Carbo monoterapia;4=Carbo-Taxol-Beva;5=otro;6=chemotherapy, but regimen unknown;7=no chemotherapy;NA=no information',
    'MMR GERMLINE STATUS':'0=UNKNOWN, 1=MUTATED, 2= UNMUTATED',
    'VITAL STATUS':'0=alive; 1=dead',
    'OS_CNIO':'Overall survival in days',
    'DATE USED FOR OS':'D=diagnosis date used for OS, NA=unknown',
    'PARTIAL DATE DEATH_LASTv':'YES= partial date xx/xx/y or xx/m/y, NO= Complete date or NA',
    'PARTIAL DATE DIAGNOSIS':'YES= partial date xx/xx/y or xx/m/y, NO= Complete date or NA',
    'FAMILIAL ANTECEDENTS OF  COLON, ENDOMETRIAL AND/OR OVARIAN CANCER':'0=NO;1=YES;NA=unknown'
}])

# Eliminate previous row with index 0 and incomplete coding
rvb_final = rvb_final.drop(0).reset_index(drop=True)
# Concatenate this row with the dataframe
rvb_final = pd.concat([coding_row, rvb_final], ignore_index=True)
# Print the final dataframe
print(rvb_final.head(6))
# Save the dataframe to an excel file. 
rvb_final.to_excel('/home/vant/TFM/RVB_bbdd_filered1.xlsx',index=False)

  ID CNIO_MJG                                HISTOLOGY_coded MJG  \
0         NaN  1=serous, 2=mucinous, 3=endometrioid, 4=clear ...   
1        RVB1                                                NaN   
2        RVB2                                                  3   
3        RVB3                                                  5   
4        RVB4                                                  1   

   Age at diagnosis                                        TUMOR GRADE  \
0               NaN  1=well differentiated, 2=moderately differenti...   
1              46.0                                                NaN   
2              47.0                                                  1   
3              35.0                                                NaN   
4              83.0                            ovario:3; ENDOMETRIO: 1   

   FIGO STAGE_MJG  FIGOa FIGOL    FIRST LINE or adjuvant TREATMENT_MJG  \
0             NaN    NaN    NA  CHEMOTHERAPY NO=0, YES=1, NA=NOT KNOWN  